Imports

In [1]:
import os
from PIL import Image

Function to check if 2 colors are in range 

In [2]:
def ValuesInRange(value1, value2, range):
    return abs(value1 - value2) <= range

def ColorsInRange(color1, color2, range):
    r1, g1, b1 = color1
    r2, g2, b2 = color2
    return ValuesInRange(r1, r2, range) and ValuesInRange(g1, g2, range) and ValuesInRange(b1, b2, range)

Function to represent index map as an image

In [3]:
def GetMaxValue(index_map):
    max_value = 0
    for column in index_map:
        for value in column:
            if value > max_value:
                max_value = value
    return max_value

def NormalizeIndexMap(index_map, max_value):
    normalized_index_map = []
    for column in index_map:
        new_column = []
        for value in column:
            new_value = round(value / max_value * 255)
            new_column.append(new_value)
        normalized_index_map.append(new_column)
    return normalized_index_map

def IndexMapToImage(index_map):
    max_value = GetMaxValue(index_map)
    normalized_index_map = NormalizeIndexMap(index_map, max_value)
    x, y = (len(index_map), len(index_map[0]))
    normalized_index_map_image = Image.new("L", (x, y))
    for i in range(x):
        for j in range(y):
            value = normalized_index_map[i][j]
            normalized_index_map_image.putpixel((i, j), value)
    return normalized_index_map_image

CalculateColorMap function

In [4]:
def CalculateColorMap(image):
    color_map = {}
    x, y = image.size
    for i in range(x):
        for j in range(y):
            r, g, b = image.getpixel((i, j))
            if (not (r, g, b) in color_map):
                color_map[(r, g, b)] = len(color_map)
    index_map = []
    for i in range(x):
        column = []
        for j in range(y):
            r, g, b = image.getpixel((i, j))
            index = color_map.get((r, g, b))
            column.append(index)
        index_map.append(column)
    return (index_map, color_map)

QuantizationLevels function

In [5]:
def QuantizationLevels(color_map, range):
    color_map_copy = color_map.copy()
    quantized_color_map = {}
    removed_colors = {}
    for color1 in color_map.keys():
        if color_map_copy[color1] != -1:
            quantized_color_map[color1] = color_map[color1]
            color_map_copy[color1] = -1
            for color2 in color_map.keys():
                if color_map_copy[color2] != -1 and ColorsInRange(color1, color2, range):
                    removed_colors[color2] = (color_map[color2], color_map[color1])
                    color_map_copy[color2] = -1                    
    return (quantized_color_map, removed_colors)

AdjustIndex function

In [6]:
def AdjustIndex(index_map, removed_colors):
    removed_colors_indices = {old_index: new_index for old_index, new_index in removed_colors.values()}
    modified_index_map = []
    for column in index_map:
        new_column = []
        for index in column:
            if index in removed_colors_indices:
                new_column.append(removed_colors_indices[index])
            else:
                new_column.append(index)
        modified_index_map.append(new_column)
    return modified_index_map

ReverseColorMap function

In [7]:
def ReverseColorMap(color_map):
    return {value:key for key, value in color_map.items()}

ColorMapToImage function

In [8]:
def ColorMapToImage(index_map, color_map):
    color_map = ReverseColorMap(color_map)
    x = len(index_map)
    y = len(index_map[0])
    image = Image.new("RGB", (x, y))
    for i in range(x):
        for j in range(y):
            index = index_map[i][j]
            r, g, b = color_map.get(index)
            image.putpixel((i,j), (r, g, b))
    return image

Test

In [9]:
current_directory = os.getcwd()
image_path = current_directory + R"\images\test.jpg"
image = Image.open(image_path)

In [10]:
index_map, color_map = CalculateColorMap(image)

In [11]:
reconstructed_image = ColorMapToImage(index_map, color_map)
reconstructed_image.save(current_directory + R"\results\test_reconstructed.jpg")

In [12]:
index_map_image = IndexMapToImage(index_map)
index_map_image.save(current_directory + R"\results\test_reconstructed_index_map.jpg")

In [17]:
quantized_color_map, removed_colors = QuantizationLevels(color_map, 30)

In [18]:
modified_index_map = AdjustIndex(index_map, removed_colors)

In [19]:
quantized_image = ColorMapToImage(modified_index_map, quantized_color_map)
quantized_image.save(current_directory + R"\results\test_quantized_30.jpg")

In [20]:
modified_index_map_image = IndexMapToImage(modified_index_map)
modified_index_map_image.save(current_directory + R"\results\test_quantized_30_index_map.jpg")